Funcion que despues escribe en un excel el ejemplo

In [1]:
function write_response(name,model,x,y)
    # Escribir el nombre del experimento
    println(name)
    # Escribir el resultado de la solucion primal
    p_s=BilevelJuMP.primal_status(model)
    println("Primal Status: $p_s")
    # Escribir el codigo y tipo de terminacion
    t_s=BilevelJuMP.termination_status(model)
    println("Termination Status: $t_s")
    # Objetive value
    o_v=BilevelJuMP.objective_value(model)
    println("Objetive Value $o_v")
    # Lider Vars Value
    l_vars=BilevelJuMP.value.(x)
    println("Lider Vars: $l_vars")
    # Follower Vars Value
    f_vars=BilevelJuMP.value.(y)
    println("Followers Vars Value $f_vars")
    
end

write_response (generic function with 1 method)

In [35]:
function get_optimal(model)
    resultado=@benchmark optimize!(model)
    return resultado,model
end

get_optimal (generic function with 1 method)

# Importar los modulos necesarios (BilevelJuMP y los optimizadores)

In [2]:
using BilevelJuMP, HiGHS, Ipopt, SCIP


# Crear el modelo del experimento

In [3]:
model = BilevelModel()

An Abstract JuMP Model
Feasibility problem with:
Variables: 0
Upper Constraints: 0
Lower Constraints: 0
Bilevel Model
Solution method: BilevelJuMP.NoMode{Float64}
No solver attached

In [4]:
BilevelJuMP.@variable(Lower(model), x)
BilevelJuMP.@variable(Upper(model), y)

y

In [5]:
BilevelJuMP.@objective(Upper(model), Min, 3x + y)
BilevelJuMP.@constraints(Upper(model), begin
    x <= 5
    y <= 8
    y >= 0
end)

BilevelJuMP.@objective(Lower(model), Min, -x)
BilevelJuMP.@constraints(Lower(model), begin
    c1,  x +  y <= 8
    c2, 4x +  y >= 8
    c3, 2x +  y <= 13
    c4, 2x - 7y <= 0
end)

(c1 : x + y ≤ 8, c2 : 4 x + y ≥ 8, c3 : 2 x + y ≤ 13, c4 : 2 x - 7 y ≤ 0)

## Using HiGHS Optimizer

In [6]:
BilevelJuMP.set_optimizer(model, HiGHS.Optimizer)

BilevelJuMP.set_mode(model,
    BilevelJuMP.BigMMode(primal_big_M = 100, dual_big_M = 100))

BilevelJuMP.optimize!(model)

BilevelJuMP.objective_value(model)
@assert abs(BilevelJuMP.objective_value(model) - (3 * (7/2 * 8/15) + 8/15)) < 1e-1 # src

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 9e+02]
  Cost   [1e+00, 3e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+02]
Assessing feasibility of MIP using primal feasibility and integrality tolerance of       1e-06
Solution has               num          max          sum
Col     infeasibilities      0            0            0
Integer infeasibilities      0            0            0
Row     infeasibilities      4           13           30
Row     residuals            0            0            0
Attempting to find feasible solution by solving LP for user-supplied values of discrete variables
Coefficient ranges:
  Matrix [1e+00, 9e+02]
  Cost   [1e+00, 3e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+02]
Presolving model
13 rows, 6 cols, 24 nonzeros  0s
Problem status detected on presolve: Infeasible
Model status        : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :      

# Using SOS1 

In [33]:
set_optimizer(model, SCIP.Optimizer)

BilevelJuMP.set_mode(model, BilevelJuMP.SOS1Mode())

optimize!(model)

objective_value(model)

@assert abs(objective_value(model) - (3 * (3.5 * 8/15) + 8/15)) < 1e-1 # src

presolving:
(round 1, fast)       0 del vars, 3 del conss, 0 add conss, 3 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 3 del conss, 0 add conss, 7 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       0 del vars, 3 del conss, 0 add conss, 8 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, exhaustive) 0 del vars, 6 del conss, 0 add conss, 8 chg bounds, 3 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation skipped: there exist constraints that cannot be handled by symmetry methods.
presolving (5 rounds: 5 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 6 deleted constraints, 0 added constraints, 8 tightened bounds, 0 added holes, 3 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 10 variables (0 bin, 0 int, 0 impl, 10 cont) and 10 constraints
      4 constraints of type <SOS1>
      6 constrain

# Using Product Mode

In [8]:
set_optimizer(model, Ipopt.Optimizer)

BilevelJuMP.set_mode(model, BilevelJuMP.ProductMode())

optimize!(model)

objective_value(model)

@assert abs(objective_value(model) - (3 * (7/2 * 8/15) + 8/15)) < 1e-1 # src


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:       30
Number of nonzeros in Lagrangian Hessian.............:        8

Total number of variables............................:        6
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        3
Total number of equality constraints.................:        1
Total number of inequality co

In [24]:
using BenchmarkTools

resultado=@benchmark optimize!(model)

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:       30
Number of nonzeros in Lagrangian Hessian.............:        8

Total number of variables............................:        6
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        3
Total number of equality constraints.................:        1
Total number of inequality constraints...............:       11
        inequality constraints with only lower bounds:        2
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        9

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 8.00e+00 1.25e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

BenchmarkTools.Trial: 77 samples with 1 evaluation.
 Range (min … max):  34.212 ms … 117.528 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     55.987 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   62.431 ms ±  21.665 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁ ▃  ▁    █▃    ▁▃   ▁                                        
  █▄█▁▄█▄▄▇▇██▇▇▄▇██▄▄▇█▇▁▁▄▁▄▁▄▄▇▁▇▇▄▄▁▇▁▁▁▄▁▄▁▁▁▁▁▁▇▁▁▇▄▇▁▄▄ ▁
  34.2 ms         Histogram: frequency by time          110 ms <

 Memory estimate: 182.45 KiB, allocs estimate: 3884.

In [25]:
resultado

BenchmarkTools.Trial: 77 samples with 1 evaluation.
 Range (min … max):  34.212 ms … 117.528 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     55.987 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   62.431 ms ±  21.665 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁ ▃  ▁    █▃    ▁▃   ▁                                        
  █▄█▁▄█▄▄▇▇██▇▇▄▇██▄▄▇█▇▁▁▄▁▄▁▄▄▇▁▇▇▄▄▁▇▁▁▁▄▁▄▁▁▁▁▁▁▇▁▁▇▄▇▁▄▄ ▁
  34.2 ms         Histogram: frequency by time          110 ms <

 Memory estimate: 182.45 KiB, allocs estimate: 3884.

In [26]:
using Pkg
Pkg.add("XLSX")
Pkg.add("BenchmarkTools")
Pkg.add("DataFrames")  # Para trabajar con DataFrames

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed XML2_jll ──────────────────── v2.13.5+0
   Installed TableTraits ───────────────── v1.0.1
   Installed DataAPI ───────────────────── v1.16.0
   Installed ArgCheck ──────────────────── v2.4.0
   Installed ZipArchives ───────────────── v2.4.0
   Installed EzXML ─────────────────────── v1.2.0
   Installed Tables ────────────────────── v1.12.0
   Installed InputBuffers ──────────────── v1.0.0
   Installed ZipFile ───────────────────── v0.10.1
   Installed DataValueInterfaces ───────── v1.0.0
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed Libiconv_jll ──────────────── v1.17.0+1
   Installed CodecInflate64 ────────────── v0.1.3
   Installed XLSX ──────────────────────── v0.10.4
    Updating `~/.julia/environments/v1.8/Project.toml`
  [fdbf4ff8] + XLSX v0.10.4
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [dce04be8] + ArgCheck v2.4.0
  [6309b1aa] + CodecInflate

In [27]:
using BenchmarkTools
using XLSX
using DataFrames

function tarea()
    sleep(2)  # Simula una tarea que toma tiempo
    return "Tarea completada"
end

# Ejecutar el benchmark
resultado = @benchmark tarea()

# Extraer los tiempos y otros datos relevantes
tiempos = resultado.times  # Tiempos en nanosegundos
min_tiempo = minimum(tiempos) / 1e9  # Tiempo mínimo en segundos
max_tiempo = maximum(tiempos) / 1e9  # Tiempo máximo en segundos
promedio_tiempo = mean(tiempos) / 1e9  # Tiempo promedio en segundos

# Extraer el uso de recursos
num_asignaciones = sum(resultado.allocs)  # Total de asignaciones de memoria
memoria_usada = maximum(resultado.memory) / (1024^2)  # Memoria máxima usada en MB

# Crear un DataFrame para almacenar los resultados
df_resultados = DataFrame(
    Mínimo=min_tiempo,
    Máximo=max_tiempo,
    Promedio=promedio_tiempo,
    Asignaciones=num_asignaciones,
    Memoria_MB=memoria_usada
)

Row,Mínimo,Máximo,Promedio,Asignaciones,Memoria_MB
,Float64,Float64,Float64,Int64,Float64
1,2.00143,2.00203,2.00178,66,0.00164795


In [32]:
# Exportar el DataFrame a un archivo Excel
XLSX.writetable("resultados_benchmark.xlsx", df_resultados)

In [29]:
# Exportar múltiples tablas o resultados a diferentes hojas
XLSX.openxlsx("resultados_benchmark.xlsx", mode="w") do xf
    XLSX.writetable(xf, "Resultados", df_resultados)
end

MethodError: MethodError: no method matching writetable(::XLSX.XLSXFile, ::String, ::DataFrame)
Closest candidates are:
  writetable(!Matched::Union{AbstractString, IO}, ::Any, ::Any; overwrite, sheetname, anchor_cell) at ~/.julia/packages/XLSX/vzApD/src/write.jl:672
  writetable(!Matched::Union{AbstractString, IO}, ::Any; kw...) at ~/.julia/packages/XLSX/vzApD/src/tables_interface.jl:30

In [37]:
time,model=get_optimal(model)

presolving:
(round 1, fast)       0 del vars, 3 del conss, 0 add conss, 3 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 3 del conss, 0 add conss, 7 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       0 del vars, 3 del conss, 0 add conss, 8 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, exhaustive) 0 del vars, 6 del conss, 0 add conss, 8 chg bounds, 3 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation skipped: there exist constraints that cannot be handled by symmetry methods.
presolving (5 rounds: 5 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 6 deleted constraints, 0 added constraints, 8 tightened bounds, 0 added holes, 3 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 10 variables (0 bin, 0 int, 0 impl, 10 cont) and 10 constraints
      4 constraints of type <SOS1>
      6 constrain

(Trial(6.569 ms), An Abstract JuMP Model
Minimization problem with:
Variables: 2
Upper objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Lower objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Upper Constraints: 3
Lower Constraints: 4
Bilevel Model
Solution method: BilevelJuMP.SOS1Mode{Float64}()
Solver name: SCIP
Names registered in the model: c1, c2, c3, c4, x, y)

In [38]:
time

BenchmarkTools.Trial: 484 samples with 1 evaluation.
 Range (min … max):   6.569 ms … 30.734 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      9.823 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   10.312 ms ±  2.741 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▂▇▅▄▅█▆ ▆▅▄▆▁▇▇▂▄▄▄▁▅▁▂▁▁ ▃▂▁ ▁ ▁▁                          
  ██████████████████████████▆█████▆███▃▅▅▄▅▆▄▃█▄▃▃▁▃▁▃▁▃▁▃▁▃▃ ▅
  6.57 ms         Histogram: frequency by time        17.7 ms <

 Memory estimate: 130.38 KiB, allocs estimate: 2583.

In [39]:
model

An Abstract JuMP Model
Minimization problem with:
Variables: 2
Upper objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Lower objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Upper Constraints: 3
Lower Constraints: 4
Bilevel Model
Solution method: BilevelJuMP.SOS1Mode{Float64}()
Solver name: SCIP
Names registered in the model: c1, c2, c3, c4, x, y